## Purpose of this notebook
(Author: Siqi Xiang)

This notebook explores the customer-level snapshot dataset constructed from processed
transaction data.

The goals are to:
- Validate the churn label definition
- Examine relationships between customer behavior features and churn
- Evaluate whether simple rule-based approaches can effectively identify high-risk customers
- Decide whether training a predictive model is justified

This analysis is used to guide modeling and decision-making, not to replace pipeline logic. 

**It includes**:
- **Key Findings of relationship between features (recency, frequency, monetary) and churn rate.**
- **Conclusion of Plan A (no model)**
- **Motivation of Plan B: build a predictive model**

In [1]:
import boto3
import pandas as pd
from io import BytesIO

BUCKET = "online-retail-churn-siqi-dev"
REF_DATE = "2011-10-10"

SNAPSHOT_KEY = f"features/online_retail/dt=2026-01-12/customer_snapshot_ref={REF_DATE}.parquet"

session = boto3.Session(profile_name="siqi-dev")
s3 = session.client("s3")

obj = s3.get_object(Bucket=BUCKET, Key=SNAPSHOT_KEY)
snapshot = pd.read_parquet(BytesIO(obj["Body"].read()), engine="pyarrow")

snapshot.shape, snapshot.head()

((3013, 11),
    CustomerID last_purchase_date  frequency_180d  monetary_180d  \
 0       12347         2011-08-02               2         967.43   
 1       12348         2011-09-25               1         310.00   
 2       12352         2011-09-28               2         632.50   
 3       12353         2011-05-19               1          89.00   
 4       12354         2011-04-21               1        1079.40   
 
    days_active_180d  recency_days  aov_180d  churn_60d reference_date  \
 0                 2            69   483.715          0     2011-10-10   
 1                 1            15   310.000          1     2011-10-10   
 2                 2            12   316.250          0     2011-10-10   
 3                 1           144    89.000          1     2011-10-10   
 4                 1           172  1079.400          1     2011-10-10   
 
    lookback_days  horizon_days  
 0            180            60  
 1            180            60  
 2            180            

In [2]:
snapshot.dtypes

CustomerID                     Int64
last_purchase_date    datetime64[ns]
frequency_180d                 int64
monetary_180d                float64
days_active_180d               int64
recency_days                   int64
aov_180d                     float64
churn_60d                      int64
reference_date        datetime64[ns]
lookback_days                  int64
horizon_days                   int64
dtype: object

In [11]:
snapshot.isna().mean().sort_values(ascending=False).head(20)

CustomerID            0.0
last_purchase_date    0.0
frequency_180d        0.0
monetary_180d         0.0
days_active_180d      0.0
recency_days          0.0
aov_180d              0.0
churn_60d             0.0
reference_date        0.0
lookback_days         0.0
horizon_days          0.0
recency_bin           0.0
monetary_bin          0.0
dtype: float64

In [4]:
snapshot["churn_60d"].value_counts()

churn_60d
0    1599
1    1414
Name: count, dtype: int64

In [5]:
snapshot["churn_60d"].mean()

np.float64(0.46929970129439097)

In [6]:
snapshot[["recency_days", "frequency_180d", "monetary_180d", "aov_180d"]].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

,recency_days,frequency_180d,monetary_180d,aov_180d
count,3013.000000,3013.000000,3013.000000,3013.000000
mean,55.871225,2.840358,1381.807731,413.696082
std,50.423591,4.260532,5547.299634,634.953672
min,0.000000,1.000000,2.900000,2.900000
50%,36.000000,2.000000,539.720000,306.030000
75%,91.000000,3.000000,1206.360000,457.700000
90%,138.000000,6.000000,2360.320000,736.544167
95%,157.000000,8.000000,3696.846000,1029.770000
99%,176.000000,18.000000,13666.032800,2444.715600
max,180.000000,93.000000,157867.450000,21535.900000


In [10]:
#Biger recency, bigger churn rate?

bins = [0, 7, 14, 30, 60, 90, 120, 180]
snapshot["recency_bin"] = pd.cut(snapshot["recency_days"], bins=bins, include_lowest=True)

recency_churn = (
    snapshot.groupby("recency_bin")["churn_60d"]
    .agg(churn_rate="mean", n="size")
    .reset_index()
)

recency_churn

/var/folders/df/pcptsbz12f76cfxt2hmf70tm0000gn/T/ipykernel_9527/2888123881.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  snapshot.groupby("recency_bin")["churn_60d"]


,recency_bin,churn_rate,n
0,"(-0.001, 7.0]",0.383117,462
1,"(7.0, 14.0]",0.387879,330
2,"(14.0, 30.0]",0.376812,552
3,"(30.0, 60.0]",0.428571,546
4,"(60.0, 90.0]",0.480874,366
5,"(90.0, 120.0]",0.615646,294
6,"(120.0, 180.0]",0.669546,463


In [ ]:
#Higher frequency, lower churn rate?

freq_churn = (
    snapshot.groupby("frequency_180d")["churn_60d"]
    .agg(churn_rate="mean", n="size")
    .reset_index()
    .sort_values("frequency_180d")
)

freq_churn.head(20)

,frequency_180d,churn_rate,n
0,1,0.641912,1360
1,2,0.448625,691
2,3,0.397476,317
3,4,0.250000,224
4,5,0.175439,114
5,6,0.166667,60
6,7,0.125000,72
7,8,0.105263,38
8,9,0.037037,27
9,10,0.117647,17


In [9]:
#Higher monetary, lower churn rate?

snapshot["monetary_bin"] = pd.qcut(snapshot["monetary_180d"], q=10, duplicates="drop")

monetary_churn = (
    snapshot.groupby("monetary_bin")["churn_60d"]
    .agg(churn_rate="mean", n="size")
    .reset_index()
)

monetary_churn

/var/folders/df/pcptsbz12f76cfxt2hmf70tm0000gn/T/ipykernel_9527/1756435351.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  snapshot.groupby("monetary_bin")["churn_60d"]


,monetary_bin,churn_rate,n
0,"(2.899, 143.822]",0.695364,302
1,"(143.822, 225.3]",0.657807,301
2,"(225.3, 311.828]",0.544850,301
3,"(311.828, 405.992]",0.578073,301
4,"(405.992, 539.72]",0.576159,302
5,"(539.72, 728.324]",0.475083,301
6,"(728.324, 980.064]",0.365449,301
7,"(980.064, 1396.506]",0.375415,301
8,"(1396.506, 2360.32]",0.259136,301
9,"(2360.32, 157867.45]",0.165563,302


### Key Findings: Relationship Between Features and Churn

**Recency (days since last purchase)**
- Churn rate increases monotonically as recency grows.
- Customers inactive for more than 90 days show churn rates above 60%, compared to ~38% for customers active within the last 30 days.
- This indicates that inactivity duration is a strong early warning signal for churn.

**Frequency (number of orders in past 180 days)**
- Churn rate decreases sharply as purchase frequency increases.
- Customers with only one purchase in the lookback window have a churn rate above 60%, while customers with more than 5 purchases rarely churn.
- This suggests that repeated purchasing behavior reflects habit formation and loyalty.

**Monetary (total spend in past 180 days)**
- Churn rate is significantly higher among low-spend customers and decreases steadily across higher spending segments.
- The lowest spending decile has a churn rate close to 70%, while the highest decile drops below 20%.
- Higher economic investment correlates with stronger customer retention.

**Overall conclusion**
- Recency, frequency, and monetary value all show strong and intuitive relationships with churn.
- These features alone already provide meaningful separation between high-risk and low-risk customers, indicating that churn is predictable even with simple rules.

In [ ]:
df = snapshot.copy()
df["recency_rank"] = df["recency_days"].rank(pct=True)                  
df["frequency_rank"] = 1 - df["frequency_180d"].rank(pct=True)          
df["monetary_rank"] = 1 - df["monetary_180d"].rank(pct=True)            

# Weights can be adjusted based on feature importance
df["risk_score_rule"] = 0.5*df["recency_rank"] + 0.25*df["frequency_rank"] + 0.25*df["monetary_rank"]

df[["risk_score_rule"]].describe()

,risk_score_rule
count,3013.000000
mean,0.500000
std,0.232730
min,0.007302
25%,0.321440
50%,0.493943
75%,0.697602
max,0.938807


In [13]:
def precision_at_k(df, score_col, label_col, k):
    topk = df.sort_values(score_col, ascending=False).head(k)
    return topk[label_col].mean()

base_rate = df["churn_60d"].mean()
base_rate

np.float64(0.46929970129439097)

In [14]:
for k in [100, 200, 500, 1000]:
    p = precision_at_k(df, "risk_score_rule", "churn_60d", k)
    lift = p / base_rate if base_rate > 0 else None
    print(f"K={k:4d}  precision={p:.3f}  lift={lift:.2f}")

K= 100  precision=0.770  lift=1.64
K= 200  precision=0.690  lift=1.47
K= 500  precision=0.678  lift=1.44
K=1000  precision=0.635  lift=1.35


### Plan A: Rule-based approach (if we do not use a model)

Based on exploratory analysis, customer churn shows strong and consistent relationships
with recency, frequency, and monetary value.

A simple rule-based risk score was constructed using normalized ranks of these features.
This rule-based ranking achieves meaningful lift over random selection:

- Precision@100 ≈ 0.77 (lift ≈ 1.64)
- Precision@500 ≈ 0.68 (lift ≈ 1.44)

This indicates that even without a trained model, we can already identify a high-risk
subset of customers significantly more likely to churn.

**Conclusion:**  
For an initial deployment or business pilot, a rule-based ranking is sufficient to
support capacity-limited retention actions (e.g., targeting top-K customers with
discounts or outreach). This approach is interpretable, stable, and easy to operationalize.

### Plan B: Motivation for building a predictive model

While the rule-based approach performs well, several signals suggest that a predictive
model may further improve ranking quality:

- Recency, frequency, and monetary value interact non-linearly.
- The relative importance of features may vary across customer segments.
- A trained model can learn optimal weights and interactions automatically.

Therefore, a baseline predictive model (e.g., logistic regression) will be trained
to evaluate whether it can outperform the rule-based ranking in terms of lift@K.